In [1]:
!pip install scikit-learn
!pip install nltk
!pip install contractions
!pip install pandas
!pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
RE_PATTERNS = {

    ' fuck ':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*',
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'

        ],

    ' crap ':
        [
            ' (c)(r|[^a-z0-9 ])(a|[^a-z0-9 ])(p|[^a-z0-9 ])([^ ])*',
            ' (c)([^a-z]*)(r)([^a-z]*)(a)([^a-z]*)(p)',
            ' c[!@#\$%\^\&\*]*r[!@#\$%\^&\*]*p', 'cr@p', ' c r a p',

        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$'
                                                           '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ],

    ' ass hole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole'
        ],

    ' bitch ':
        [
            'bitches', ' b[w]*i[t]*ch', ' b!tch',
            ' bi\+ch', ' b!\+ch', ' (b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            ' biatch', ' bi\*\*h', ' bytch', 'b i t c h'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' transgender':
        [
            'transgender'
        ],

    ' gay ':
        [
            'gay', 'homo'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'd i c k'
        ],

    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bull shit ':
        [
            'bullsh\*t', 'bull\$hit', 'bull sh.t'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots' 'i d i o t'
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t', 'sh\*tty',
            'sh\*ty', 'sh\*t'
        ],

    ' shit hole ':
        [
            'shythole', 'sh.thole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            'raped'
        ],

    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],

    ' ass head':
        [
            'butthead'
        ],

    ' sex ':
        [
            'sexy', 's3x', 'sexuality'
        ],

    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            ' stfu' '^stfu'
        ],

    ' for your fucking information':
        [
            ' fyfi', '^fyfi'
        ],
    ' get the fuck off':
        [
            'gtfo', '^gtfo'
        ],

    ' oh my fucking god ':
        [
            ' omfg', '^omfg'
        ],

    ' what the hell ':
        [
            ' wth', '^wth'
        ],

    ' what the fuck ':
        [
            ' wtf', '^wtf'
        ],
    ' son of bitch ':
        [
            ' sob ', '^sob '
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses', '(p)(u|[^a-z0-9 ])(s|[^a-z0-9 ])(s|[^a-z0-9 ])(y)',
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' mother fucker':
        [
            ' motha f', ' mother f', 'motherucker', ' mofo', ' mf ',
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],

    ' haha ':
        [
            'ha\*\*\*ha',
        ],
    # ' what the fuck ':
    # [
    #     ' wtf',
    # ],
}


In [3]:
from pathlib import Path

import datasets
import pandas as pd
import random

import contractions
import re, string
from nltk import word_tokenize, sent_tokenize, download
from nltk.corpus import stopwords

download('stopwords')
download('punkt')
download('punkt_tab')


def prepare(raw_data: Path) -> datasets.Dataset:
    dataset = pd.read_excel(raw_data)
    dataset = dataset.dropna()
    
    dataset["message"] = dataset["message"].apply(lambda sample: process(sample))
    dataset["message"] = dataset["message"].apply(lambda x: ' '.join(x))
    dataset["message"] = dataset["message"][dataset["message"].apply(len) > 0]
    dataset = dataset.drop_duplicates(subset=["message"])
    dataset = dataset.dropna()
    print(dataset)
    return datasets.Dataset.from_pandas(dataset)


def load_dataset(path: Path) -> datasets.Dataset:
    return datasets.load_from_disk(str(path))


def save_dataset(dataset: datasets.Dataset, path: Path) -> None:
    dataset.save_to_disk(str(path))
    

url_regex = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
def process(data):
    data = url_regex.sub(" ", data) # urls    
    data = data.lower()

    
    data = re.compile(r"(.)\1{2,}", re.DOTALL).sub(r"\1", data) # remove repeated
    
    for target, patterns in RE_PATTERNS.items():
        for pat in patterns:
            data = re.sub(pat, target, data)
            
    data = re.sub(r"[^a-z' ]", ' ', data)
    
    data = contractions.fix(data)
    data = re.compile('([^\s\w]|_)+').sub(' ', data)

    
    data = word_tokenize(data)
    return [word for word in data if word not in stopwords.words('english')]


[nltk_data] Downloading package stopwords to /home/mark/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mark/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/mark/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
import argparse
from pathlib import Path

dataset = prepare("./code-review-dataset-full.xlsx")
save_dataset(dataset, "./prepared-dataset")

                                                 message  is_toxic
0                               assignments also removed         0
1                               flavor id self flavor id         0
2                             bool session adopted false         0
3      nit starting c could done directly declaration...         0
4      confused tar process checking exited need kill...         0
...                                                  ...       ...
19643                             love great fix factory         0
19644                                     page fantastic         0
19646   amazing bet contributed lot slowdown nice rhymes         0
19647                                        great catch         0
19650  wait seeing nuances affect numbers cock pit go...         0

[12517 rows x 2 columns]


Saving the dataset (0/1 shards):   0%|          | 0/12517 [00:00<?, ? examples/s]